# Nonlinear analysis -Assignment 1

21.09.2023 - Laure Toullier

In [21]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('bmh')
import math

np.set_printoptions(precision=3)

Params = plt.rcParams
Params['figure.figsize'] = (14, 7) 

### Exercise 1

In [22]:
# Element connectivity 
connectivity = np.array([[1, 2], [2, 3]])

# Unit stiffness matrix of bar in local coordinate system
K_unit = np.array([[1, 0, -1, 0], 
                   [0, 0,  0, 0], 
                   [-1, 0, 1, 0], 
                   [0, 0, 0, 0]])

# Input data and element stiffness matrices
E = 200e3  # N/mm^2

a=3000  #m
b=4000  #m


In [23]:
#Load
P = 150*10**3 #N
theta = 30 #°

Px=P*np.cos(np.radians(theta)) #N
Py=P*np.sin(np.radians(theta)) #N

print(Px,Py)

129903.8105676658 74999.99999999999


Bar 1

In [24]:
A1 = 10000 #mm²
L1 = np.sqrt(a**2+b**2) #mm

theta1 = np.arctan(b/a)
c = np.cos(theta1)
s = np.sin(theta1)

#Transformation matrix
T = np.array([[c, s, 0, 0], 
              [-s, c, 0, 0], 
              [0, 0, c, s], 
              [0, 0, -s, c]])

k1 = E*A1/L1
k1_global = k1 * T.T @ K_unit @ T

print(k1_global)

[[ 144000.  192000. -144000. -192000.]
 [ 192000.  256000. -192000. -256000.]
 [-144000. -192000.  144000.  192000.]
 [-192000. -256000.  192000.  256000.]]


Bar 2

In [25]:
A2 = 15000 #mm²
L2 = np.sqrt(a**2+b**2) #mm

theta2 = -np.arctan(b/a)
c = np.cos(theta2)
s = np.sin(theta2)

#Transformation matrix
T = np.array([[c, s, 0, 0], 
              [-s, c, 0, 0], 
              [0, 0, c, s], 
              [0, 0, -s, c]])

k2 = E*A2/L2
k2_global = k2 * T.T @ K_unit @ T

print(k2_global)

[[ 216000. -288000. -216000.  288000.]
 [-288000.  384000.  288000. -384000.]
 [-216000.  288000.  216000. -288000.]
 [ 288000. -384000. -288000.  384000.]]


Stiffness matrix in global system

In [26]:
# Assemble global stiffness matrix
NDoF = 6
K_global = np.zeros((NDoF, NDoF))
K_element_6DOF = []

# Assemble 1st element
K1_global = np.zeros((NDoF, NDoF))
K1_global[0:4, 0:4] = k1_global[0:4, 0:4]

# Assemble 2nd element
K2_global = np.zeros((NDoF, NDoF))
K2_global[2:6, 2:6] = k2_global[0:4,0:4]

#Stiffness matrix of the entire system: K=K1+K2
K_global=K1_global+K2_global

print(K_global)

[[ 144000.  192000. -144000. -192000.       0.       0.]
 [ 192000.  256000. -192000. -256000.       0.       0.]
 [-144000. -192000.  360000.  -96000. -216000.  288000.]
 [-192000. -256000.  -96000.  640000.  288000. -384000.]
 [      0.       0. -216000.  288000.  216000. -288000.]
 [      0.       0.  288000. -384000. -288000.  384000.]]


Equilibrium of system : KU=R

Condensation of system

In [27]:
#Joints displacements
F_red = np.array([Px, Py]) # N
K_red = K_global[2:4, 2:4]
u_red = np.linalg.inv(K_red) @ F_red
print(u_red)

[0.408 0.178]


In [28]:
#Support reactions @node 1 and 3
u_vec = np.array([0, 0, u_red[0], u_red[1],0,0])
F = K_global @ u_vec

F1=K_global[0:6,2:4]@u_red

print(F)

[ -93076.905 -124102.54   129903.811   75000.     -36826.905   49102.54 ]


In [29]:
#Axial forces in each bar

F1=K1_global@u_vec
F2=K2_global@u_vec

""""""""""""""""""""""""""""""""""""""


''

### Exercise 2

#### Geometry

In [30]:
#Nodes
DDL=7
a=3000 #mm
b=4000 #mm

nodes=np.zeros(DDL*2).reshape(DDL,2)

for i in range(DDL):
    nodes[i,0]=a*i
    if i%2==0:
        nodes[i,1]=0
    else:
        nodes[i,1]=b

print(nodes)

[[    0.     0.]
 [ 3000.  4000.]
 [ 6000.     0.]
 [ 9000.  4000.]
 [12000.     0.]
 [15000.  4000.]
 [18000.     0.]]


In [31]:
#Connectivity

connectivity=[]

for i in range(DDL):
    if i<(DDL-2):
        connectivity.append([i+1, i+2])
        connectivity.append([i+1, i+3])
    if i==DDL-2:
        connectivity.append([i+1, i+2])
        
Connectivity=np.array(connectivity)
    
print(Connectivity)

[[1 2]
 [1 3]
 [2 3]
 [2 4]
 [3 4]
 [3 5]
 [4 5]
 [4 6]
 [5 6]
 [5 7]
 [6 7]]


In [32]:
# Area

A=10*10**3 #mm²     #members area equal in the entire truss
nb_members=int(np.size(Connectivity)/2)
area=np.zeros(nb_members)

for i in range(np.size(area)):
    area[i]=A
    
area

array([10000., 10000., 10000., 10000., 10000., 10000., 10000., 10000.,
       10000., 10000., 10000.])

In [33]:
#Length of each element

length=[]

for i in range(nb_members):
    x1=nodes[Connectivity[i,0]-1,0]      #For each node at the extremity of each bar, we get its location in x and y.
    y1=nodes[Connectivity[i,0]-1,1]
    x2=nodes[Connectivity[i,1]-1,0]
    y2=nodes[Connectivity[i,1]-1,1]
    
    x=x2-x1
    y=y2-y1
    length.append(np.sqrt(x**2+y**2))

length

[5000.0,
 6000.0,
 5000.0,
 6000.0,
 5000.0,
 6000.0,
 5000.0,
 6000.0,
 5000.0,
 6000.0,
 5000.0]

#### Stiffness

In [34]:
#Local stiffness matrix

k_local=E*area/length
k_local

array([400000.   , 333333.333, 400000.   , 333333.333, 400000.   ,
       333333.333, 400000.   , 333333.333, 400000.   , 333333.333,
       400000.   ])

In [35]:
# Unit stiffness matrix of bar in local coordinate system
K_unit = np.array([[1, 0, -1, 0], 
                   [0, 0,  0, 0], 
                   [-1, 0, 1, 0], 
                   [0, 0, 0, 0]])

In [36]:
#Stiffness matrix for each bar in global coordinates

angles=[]

for i in range(nb_members):
    x1=nodes[Connectivity[i,0]-1,0]      #For each node at the extremity of each bar, we get its location in x and y.
    y1=nodes[Connectivity[i,0]-1,1]
    x2=nodes[Connectivity[i,1]-1,0]
    y2=nodes[Connectivity[i,1]-1,1]
    
    if np.isclose(y1, y2):
        if np.isclose(x1, x2):
            angles.append(90)
        else:
            angles.append(0)
    else:
        theta=np.arcsin((y2-y1)/length[i]) 
        angles.append(math.degrees(theta))
    
print(angles)


[53.13010235415599, 0, -53.13010235415599, 0, 53.13010235415599, 0, -53.13010235415599, 0, 53.13010235415599, 0, -53.13010235415599]


In [37]:
#Transformation matrix for each bar

c=[]
s=[]

for i in range(nb_members):
    c.append(np.cos(np.radians(i)))
    s.append(np.sin(np.radians(i)))

T=np.zeros((nb_members,4,4))

for i in range(nb_members):
    T[i,0,0]=c[i]
    T[i,0,1]=s[i]
    T[i,1,0]=-s[i]
    T[i,1,1]=c[i]
    T[i,2,2]=c[i]
    T[i,2,3]=s[i]
    T[i,3,2]=-s[i]
    T[i,3,3]=c[i]

T[2]


array([[ 0.999,  0.035,  0.   ,  0.   ],
       [-0.035,  0.999,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.999,  0.035],
       [ 0.   ,  0.   , -0.035,  0.999]])

In [38]:
#Stiffness for each bar in global coordinates system

k_global=[]

for i in range(nb_members):
    k=k_local[i]*T[i].T@K_unit@T[i]
    k_global.append(k)

print(k_global)

[array([[ 400000.,       0., -400000.,       0.],
       [      0.,       0.,       0.,       0.],
       [-400000.,       0.,  400000.,       0.],
       [      0.,       0.,       0.,       0.]]), array([[ 3.332e+05,  5.817e+03, -3.332e+05, -5.817e+03],
       [ 5.817e+03,  1.015e+02, -5.817e+03, -1.015e+02],
       [-3.332e+05, -5.817e+03,  3.332e+05,  5.817e+03],
       [-5.817e+03, -1.015e+02,  5.817e+03,  1.015e+02]]), array([[ 399512.81 ,   13951.295, -399512.81 ,  -13951.295],
       [  13951.295,     487.19 ,  -13951.295,    -487.19 ],
       [-399512.81 ,  -13951.295,  399512.81 ,   13951.295],
       [ -13951.295,    -487.19 ,   13951.295,     487.19 ]]), array([[ 332420.316,   17421.411, -332420.316,  -17421.411],
       [  17421.411,     913.017,  -17421.411,    -913.017],
       [-332420.316,  -17421.411,  332420.316,   17421.411],
       [ -17421.411,    -913.017,   17421.411,     913.017]]), array([[ 398053.614,   27834.62 , -398053.614,  -27834.62 ],
       [  27834.62

In [40]:
#Stiffness in global system

K_global=np.zeros((DDL*2,DDL*2))
k_global = np.array(k_global)

for i, c in enumerate(Connectivity):
    u, v = c
    K_global[2*u-2:2*u,2*u-2:2*u] += k_global[i, 0:2, 0:2]
    K_global[2*u-2:2*u,2*v-2:2*v] += k_global[i, 0:2, 2:4]
    K_global[2*v-2:2*v,2*u-2:2*u] += k_global[i, 2:4, 0:2]
    K_global[2*v-2:2*v,2*v-2:2*v] += k_global[i, 2:4, 2:4]

print(K_global.shape)
print(K_global)

(14, 14)
[[ 7.332e+05  5.817e+03 -4.000e+05  0.000e+00 -3.332e+05 -5.817e+03
   0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00
   0.000e+00  0.000e+00]
 [ 5.817e+03  1.015e+02  0.000e+00  0.000e+00 -5.817e+03 -1.015e+02
   0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00
   0.000e+00  0.000e+00]
 [-4.000e+05  0.000e+00  1.132e+06  3.137e+04 -3.995e+05 -1.395e+04
  -3.324e+05 -1.742e+04  0.000e+00  0.000e+00  0.000e+00  0.000e+00
   0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  3.137e+04  1.400e+03 -1.395e+04 -4.872e+02
  -1.742e+04 -9.130e+02  0.000e+00  0.000e+00  0.000e+00  0.000e+00
   0.000e+00  0.000e+00]
 [-3.332e+05 -5.817e+03 -3.995e+05 -1.395e+04  1.462e+06  7.654e+04
  -3.981e+05 -2.783e+04 -3.308e+05 -2.894e+04  0.000e+00  0.000e+00
   0.000e+00  0.000e+00]
 [-5.817e+03 -1.015e+02 -1.395e+04 -4.872e+02  7.654e+04  5.067e+03
  -2.783e+04 -1.946e+03 -2.894e+04 -2.532e+03  0.000e+00  0.000e+00
   0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00 -

In [20]:
# Unit stiffness matrix of bar in local coordinate system
K_unit = np.array([[1, 0, -1, 0], 
                   [0, 0,  0, 0], 
                   [-1, 0, 1, 0], 
                   [0, 0, 0, 0]])

# Input data and element stiffness matrices
E = 200e3  # N/mm^2

# Bar 1
theta1 = -45
c = np.cos(np.radians(theta1))
s = np.sin(np.radians(theta1))
T = np.array([[c, s, 0, 0], 
              [-s, c, 0, 0], 
              [0, 0, c, s], 
              [0, 0, -s, c]])
A = 10e3  # mm^2
L = np.sqrt(2)*2000  # mm
k1 = E * A / L
k1_global = k1 * T.T @ K_unit @ T

# Fill in the information for Bar 2
"""

"""

# And the same for Bar 3
"""

"""


# Assemble global stiffness matrix
NDoF = 8
K_global = np.zeros((NDoF, NDoF))
K_element_8DOF = []

# Assemble 1st element
K1_global = np.zeros((NDoF, NDoF))
K1_global[0:2, 0:2] = k1_global[0:2, 0:2]
"""

"""

# Assemble 2nd element
"""

"""

# Assemble 3rd element
"""

"""

# Calculate displacements at Node 4
F_red = np.array([-3000, -18000]) * 1000  # N
K_red = K_global[6:8, 6:8]
u_red = np.linalg.inv(K_red) @ F_red

# Calculate reaction forces at Nodes 1, 2, 3
u_vec = np.array([0, 0, 0, 0, 0, 0, u_red[0], u_red[1]])
F = K_global @ u_vec

# Calculate axial forces
F_ele = K_element_8DOF[0] @ u_vec
"""

"""

# Calculate axial forces for 2nd element

# Calculate axial forces for 3rd element

LinAlgError: Singular matrix

### Exercice 2

In [ ]:
nb = 11 #number of bars in the truss

In [ ]:
#Bars area

A = 10000 #mm² 
area = []

for i in range(nb):
    area.append(A) #area constant for each bar of the truss

In [ ]:
#Bars length

a=3#m
b=4#m

length = []

for i in range(nb):
    if (i+1)%2==0:      #if bars on the top or bottom membrane
        length.append(2*a)
    else:
        length.append(np.sqrt(a^2+b^2))
print(length)
        

In [ ]:
#Local rigidity

k_local=[]

for i in range(nb):
    k_local.append(area[i]*E/length[i])
